# Tutorial01 - Getting Started

## Overview

🚀 This tutorial demonstrates a minimal application of XRFeitoria. By the end of this tutorial, you will be able to:
- Open Blender/Unreal Engine by XRFeitoria
- Import a mesh
- Add a camera
- Render the mesh and save the image

## 1. Import XRFeitoria

After [Installing](../../index.rst#installation), import ``XRfeitoria`` by

In [ ]:
import xrfeitoria as xf

## 2. Choose engine

``XRFeitoria`` supports both ``Blender`` and ``Unreal Engine``. Choose your engine and replace the following ``engine_exec_path`` with your own engine path.

Then, initialize XRFeitoria and open the engine by ``xf.init_blender`` or by ``xf.init_unreal``.

In [ ]:
# Replace with your executable path

# Blender
# engine_exec_path = 'D:/Program Files/Blender Foundation/Blender 3.3/blender.exe'

# Unreal Engine
engine_exec_path = 'D:/Program Files/Epic Games/UE_5.1/Engine/Binaries/Win64/UnrealEditor-Cmd.exe'

In [ ]:
from pathlib import Path

exec_path_stem = Path(engine_exec_path).stem.lower()
if 'blender' in exec_path_stem:
    # Open Blender
    engine = 'blender'
    xf_runner = xf.init_blender(exec_path=engine_exec_path, background=False, new_process=True)
elif 'unreal' in exec_path_stem:
    # Unreal Engine requires a project to be opened
    # Here we use a sample project, which is downloaded from the following link
    # You can also prepare your own project
    import shutil
    from xrfeitoria.utils.downloader import download
    unreal_project_zip = download(url='https://openxrlab-share.oss-cn-hongkong.aliyuncs.com/xrfeitoria/unreal_project/XRFeitoriaUnreal_Sample.zip', 
                                    dst_dir="./tutorial01/assets/")
    shutil.unpack_archive(filename=unreal_project_zip, extract_dir='./tutorial01/assets/')

    # Open Unreal Engine
    engine = 'unreal'
    xf_runner = xf.init_unreal(exec_path=engine_exec_path, 
                                background=False, 
                                new_process=True, 
                                project_path='./tutorial01/assets/XRFeitoriaUnreal_Sample/XRFeitoriaUnreal_Sample.uproject')

✨ Now you can see a new Blender/Unreal Engine process has been started.

![](http://file.bj.zoe.sensetime.com/resources/meihaiyi/xrfeitoria/pics/tutorial/01/blender/open_blender.png)

## 3. Import a mesh

Download the [Spot](http://www.cs.cmu.edu/~kmcrane/Projects/ModelRepository/index.html#spot) model by the following cell.

In [ ]:
import shutil
from xrfeitoria.utils.downloader import download

# Download the Spot
spot_zip = download('http://file.bj.zoe.sensetime.com/resources/meihaiyi/xrfeitoria/assets/spot.zip', dst_dir="./tutorial01/assets/")
shutil.unpack_archive(filename=spot_zip, extract_dir='./tutorial01/assets/')
spot_path = Path("./tutorial01/assets/spot/spot_triangulated.obj").resolve()

Import the .obj file to create an ``Actor`` instance.

``Actor`` is the container of a mesh. By using ``Actor``, you can place a mesh in the space and set its transform data (location, rotation and scale).

In [ ]:
# Import the spot
actor_spot = xf_runner.Actor.import_from_file(file_path=spot_path)

Switch to the engine window, and you can see the spot has been imported. The space to place the ``Actor``s is called ``Level``. ``Level`` is a container of all ``Actor``s. You can add, remove, or modify ``Actor``s in the `Level`.

![](http://file.bj.zoe.sensetime.com/resources/meihaiyi/xrfeitoria/pics/tutorial/01/blender/import_spot.png)

If you use ``Unreal Engine``, the final step before rendering is to save the ``Level``.

In [ ]:
# save the level
if engine == 'unreal':
    xf_runner.utils.save_current_level()   

## 4. Add a sequence for rendering

``Sequence`` is a multifunctional class in XRFeitoria. It can be used for:
- rendering
- adding transform keys
- grouping different objects.

Here, we use it for rendering.

In [ ]:
from xrfeitoria.data_structure.models import RenderPass

# Use `with` statement to create a sequence, and it will be automatically close the sequence after the code block is executed.
with xf_runner.Sequence.new(seq_name="MySequence") as seq:

    # Add a camera and make it look at the spot
    camera_location = (-3.0, 0.0, 0.0)
    camera_rotation = xf_runner.utils.get_rotation_to_look_at(location=camera_location, target=actor_spot.location)
    seq.spawn_camera(location=camera_location, rotation=camera_rotation, fov=90)

    # Add a render job to renderer
    # In render job, you can specify the output path, resolution, render passes, etc.
    # The output path is the path to save the rendered data.
    # The resolution is the resolution of the rendered image.
    # The render passes define what kind of data you want to render, such as img, depth, normal, etc.
    # and what kind of format you want to save, such as png, exr, etc.
    seq.add_to_renderer(
        output_path=f'./tutorial01/outputs/{engine}/',
        resolution=(1280, 720),
        render_passes=[RenderPass('img', 'png')]
    )

## 5. Render

The following code renders all the render jobs and save the images to the ``output_path`` you set in ``seq.add_to_renderer`` above.

In [ ]:
xf_runner.render()

Check the ``output_path``, you can see the rendered images like this:

![](http://file.bj.zoe.sensetime.com/resources/meihaiyi/xrfeitoria/pics/tutorial/01/blender/0000.png)

## 6. Final step

🥳 This is a good start! Finally, **Do remember** to close the engine. 

In [ ]:
xf_runner.close()

Ref to [api docs](../../apis/xrfeitoria.rst), you can always use ``with`` statement to ensure the engine is closed when the codes are finished.